In [ ]:
%load_ext autoreload
%autoreload 2

from preprocessing import oversample_minority_classes
import numpy as np; np.random.seed(0)
from config import OUTPUT_DIR
from util import *
from AL import AL_split, mAP_and_APs, evaluation_dict, AL_resnet16_simulation
import pickle

In [ ]:
X = np.load(INTERMEDIATE / 'logmel_multiclass_noise.npy')
Y = np.load(INTERMEDIATE / 'logmel_labels_multiclass_noise.npy')

In [ ]:
def LC_sampling(classifier, X_unlabelled, n_instances):
    probabilities = classifier.predict_proba(X_unlabelled, batch_size=32, verbose=2)
    # least confidence = (1 - max probability)
    least_confidence = 1 - np.max(probabilities, axis=1)
    uncertain_indices = np.argsort(least_confidence)
    selected_indices = uncertain_indices[:n_instances]
    return selected_indices, X_unlabelled[selected_indices] 


classifier, metrics = AL_resnet16_simulation(
    LC_sampling, 
    X, Y, 
    working_dir=OUTPUT_DIR / 'AL' / 'LC_300_0.3_', 
    identity='LC_300_0.3_',
    budget_cap=0.3,
    n_queries=300, 
    oversample=True,
    batch=32, 
)